# Setup for google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import sys
%cd "/content/drive/MyDrive/ColabNotebooks/Audio-ex3"
sys.path.append("/content/drive/MyDrive/ColabNotebooks/Audio-ex3")

In [ ]:
!git pull

In [ ]:
!pip install -r requirements.txt
%cd src

---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

True

---
# Loading Data

In [3]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
dm = loader.AudioDataModule()
dm.setup("fit")

---
# Logger

In [4]:
from dotenv import load_dotenv
import os

load_dotenv('./../.env')
API_TOKEN = os.environ.get("API_TOKEN")
PROJECT_NAME = os.environ.get("PROJECT_NAME")

In [5]:
from lightning.pytorch.loggers import NeptuneLogger

logger_config = {
    "api_key": API_TOKEN,
    "project": PROJECT_NAME,
    "log_model_checkpoints": True,
    "tags": ["best one"]
}

---
# Create the classifier and train it

In [11]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

neptune_logger = NeptuneLogger(**logger_config)
config = classifier.default_config

trainer = classifier.train_func(config=config, dm=dm, logger=neptune_logger, num_epochs=50)

NameError: name 'dm' is not defined

---
# Test the classifier

# prepre the test data

In [6]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
dm_test = loader.AudioDataModule()
dm_test.setup("test")

# Trainer

In [5]:
from lightning.pytorch.loggers import NeptuneLogger

logger_config = {
    "api_key": API_TOKEN,
    "project": PROJECT_NAME,
    "log_model_checkpoints": False,
    "tags": ["test"]
}
neptune_logger_test = NeptuneLogger(**logger_config)

In [7]:
import lightning.pytorch as pl
trainer_test = pl.Trainer(
        devices="auto",
        accelerator="auto",
        logger=neptune_logger_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## Load the model

In [8]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

model_test = classifier.DigitClassifier.load_from_checkpoint(".neptune/AUD-174/AUD-174/checkpoints/epoch=1-step=400.ckpt", n_feature=loader.MFCC_FEATURES)

D:\Documents\GitHub\Audio-ex3\.venv\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


## Test the model

In [10]:
trainer_test.test(model_test, datamodule=dm_test)
neptune_logger_test.run.stop()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    val_avg_accuracy            0.916015625
      val_avg_loss          0.30893951654434204
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 4 operations to synchronize with Neptune. Do not kill this process.
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 0.0
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must 

---
# Validation

In [ ]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
importlib.reload(importlib.import_module("model_tuner"))
import model_tuner

model_tuner.tune_classifier(logger_config=logger_config, dm=dm)